<a href="https://colab.research.google.com/github/Madhulik123/Data-Analysis-project/blob/main/heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("Medicaldataset.csv")

In [4]:
#shape of the dataset
df.shape

(1319, 9)

In [5]:
# Datatype of each column in dataframe
df.dtypes

,0
Age,int64
Gender,int64
Heart rate,int64
Systolic blood pressure,int64
Diastolic blood pressure,int64
Blood sugar,float64
CK-MB,float64
Troponin,float64
Result,object


In [6]:
#Data Cleaning Steps
# Missing value detection
df.isna().sum()

,0
Age,0
Gender,0
Heart rate,0
Systolic blood pressure,0
Diastolic blood pressure,0
Blood sugar,0
CK-MB,0
Troponin,0
Result,0


#There is no need to deal with data missing as there is no missing data.

In [7]:
#Find % of null values in datasets
print(df.isna().sum()/len(df)*100, '\n')

Age                         0.0
Gender                      0.0
Heart rate                  0.0
Systolic blood pressure     0.0
Diastolic blood pressure    0.0
Blood sugar                 0.0
CK-MB                       0.0
Troponin                    0.0
Result                      0.0
dtype: float64 



In [8]:
#find all the symbol or character in the dataset which should not be part of the column, find for all columns

import string

# Function to find non-standard characters in a column
def find_non_standard_chars(df, column):
    # Define allowed characters (letters, numbers, whitespace, common punctuation)
    allowed_chars = string.ascii_letters + string.digits + string.whitespace + string.punctuation

    # Initialize a set to store unique non-standard characters
    non_standard_chars = set()

    # Iterate through each value in the column
    for value in df[column].dropna(): # Use dropna to skip NaN values
        # Convert value to string (handle potential non-string data types)
        value_str = str(value)
        # Check each character in the string
        for char in value_str:
            if char not in allowed_chars:
                non_standard_chars.add(char)
    return non_standard_chars

# Iterate through all columns in the DataFrame
for col in df.columns:
    non_standard_chars_in_col = find_non_standard_chars(df, col)
    if non_standard_chars_in_col:
        print(f"Non-standard characters found in column '{col}': {non_standard_chars_in_col}")
    else:
        print(f"No non-standard characters found in column '{col}' (or column is empty/contains only standard chars).")

No non-standard characters found in column 'Age' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Gender' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Heart rate' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Systolic blood pressure' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Diastolic blood pressure' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Blood sugar' (or column is empty/contains only standard chars).
No non-standard characters found in column 'CK-MB' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Troponin' (or column is empty/contains only standard chars).
No non-standard characters found in column 'Result' (or column is empty/contains only standard chars).
